In [ ]:
import asyncio
from typing import Annotated
from semantic_kernel.agents import (
    Agent, ChatCompletionAgent, ConcurrentOrchestration, ChatHistoryAgentThread
    )
from semantic_kernel.agents.runtime import InProcessRuntime
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.functions import kernel_function
import typing as t
from utils import chairAssemblyStep

from dotenv import load_dotenv
load_dotenv(override=True)

In [ ]:
class StepInferPlugin:
    """A plugin to infer a step of chair assembly."""

    @kernel_function(description="Infer a step of chair assembly.")
    def infer_step(
        self,
        step_name: Annotated[str, "Name of the assembly step"],
        test_images: Annotated[str, "A list of test images delimited by a space"],
    ) -> Annotated[str, "Returns yes or no if the images are classified into the step"]:
        # If the received value is a space-separated string, split it into a List[str]
        images: t.List[str] = test_images.split()
        return chairAssemblyStep.run(step_name, images)


In [ ]:
def get_agents() -> list[Agent]:
    """Return a list of agents that will participate in the concurrent orchestration.

    Feel free to add or remove agents.
    """
    step1_pickupbolt_agent = ChatCompletionAgent(
        name="step1_pickupbolt",
        instructions="You are determining the step of a chair assembly process. You are responsible for the step of which name is '1_pickupbolt'. Use the step name as the parameter step_name.",
        service=AzureChatCompletion(),
        plugins=[StepInferPlugin()]
    )

    step2_placeparts_agent = ChatCompletionAgent(
        name="step2_placeparts",
        instructions="You are determining the step of a chair assembly process. You are responsible for the step of which name is '2_placeparts'. Use the step name as the parameter step_name.",
        service=AzureChatCompletion(),
        plugins=[StepInferPlugin()]
    )

    step3_placebolts_agent = ChatCompletionAgent(
        name="step3_placebolts",
        instructions="You are determining the step of a chair assembly process. You are responsible for the step of which name is '3_placebolts'. Use the step name as the parameter step_name.",
        service=AzureChatCompletion(),
        plugins=[StepInferPlugin()]
    )

    step4_hexwrench_light_agent = ChatCompletionAgent(
        name="step4_hexwrench_light",
        instructions="You are determining the step of a chair assembly process. You are responsible for the step of which name is '4_hexwrench_light'. Use the step name as the parameter step_name.",
        service=AzureChatCompletion(),
        plugins=[StepInferPlugin()]
    )

    step5_hexwrench_tight_agent = ChatCompletionAgent(
        name="step5_hexwrench_tight",
        instructions="You are determining the step of a chair assembly process. You are responsible for the step of which name is '5_hexwrench_tight'. Use the step name as the parameter step_name.",
        service=AzureChatCompletion(),
        plugins=[StepInferPlugin()]
    )

    step6_attachcylinder_agent = ChatCompletionAgent(
        name="step6_attachcylinder",
        instructions="You are determining the step of a chair assembly process. You are responsible for the step of which name is '6_attachcylinder'. Use the step name as the parameter step_name.",
        service=AzureChatCompletion(),
        plugins=[StepInferPlugin()]
    )

    step7_moveseat_agent = ChatCompletionAgent(
        name="step7_moveseat",
        instructions="You are determining the step of a chair assembly process. You are responsible for the step of which name is '7_moveseat'. Use the step name as the parameter step_name.",
        service=AzureChatCompletion(),
        plugins=[StepInferPlugin()]
    )

    step8_attachleg_agent = ChatCompletionAgent(
        name="step8_attachleg",
        instructions="You are determining the step of a chair assembly process. You are responsible for the step of which name is '8_attachleg'. Use the step name as the parameter step_name.",
        service=AzureChatCompletion(),
        plugins=[StepInferPlugin()]
    )

    step9_finishedchair_agent = ChatCompletionAgent(
        name="step9_finishedchair",
        instructions="You are determining the step of a chair assembly process. You are responsible for the step of which name is '9_finishedchair'. Use the step name as the parameter step_name.",
        service=AzureChatCompletion(),
        plugins=[StepInferPlugin()]
    )

    return [step1_pickupbolt_agent, step2_placeparts_agent, step3_placebolts_agent,
            step4_hexwrench_light_agent, step5_hexwrench_tight_agent,
            step6_attachcylinder_agent, step7_moveseat_agent, step8_attachleg_agent,
            step9_finishedchair_agent]


In [ ]:
"""Main function to run the agents."""
# 1. Create a concurrent orchestration with multiple agents
agents = get_agents()
concurrent_orchestration = ConcurrentOrchestration(members=agents)

# 2. Create a runtime and start it
runtime = InProcessRuntime()
runtime.start()

# 3. Invoke the orchestration with a task and the runtime
orchestration_result = await concurrent_orchestration.invoke(
    task="The following images are a consecutive frame of a chair assembly step. " \
         "Return the result of each agent in yes or no" \
         "The list of images: frame_0053_t53.0s.jpg frame_0054_t54.0s.jpg frame_0055_t55.0s.jpg",
    runtime=runtime,
)

# 4. Wait for the results
# Note: the order of the results is not guaranteed to be the same
# as the order of the agents in the orchestration.
value = await orchestration_result.get(timeout=1800)
for item in value:
    print(f"# {item.name}: {item.content}")

# 5. Stop the runtime after the invocation is complete
await runtime.stop_when_idle()
